In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from collections import deque 
import numpy as np
import gym
import random

EPISODES = 1000
TIME_MAX = 500

In [29]:
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.001
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
        
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)
                       
        
        

In [34]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = Agent(state_size, action_size)
    done = False
    batch_size = 32
    
    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for frame in range(TIME_MAX):
            if e == (EPISODES - 1):
                env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            
            agent.remember(state, action, reward, next_state, done)
            
            state = next_state
            if done:
                if frame == (TIME_MAX - 1):
                    agent.save(name="best.h5")
                    
                print("episode: {}/{}, score: {}, epsilon: {:.2}".format(e, EPISODES, frame, agent.epsilon))
                env.render(close=True)
                break
                    
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

[2018-03-02 22:50:39,029] Making new env: CartPole-v1


episode: 0/1000, score: 14, epsilon: 1.0
episode: 1/1000, score: 28, epsilon: 1.0


episode: 2/1000, score: 30, epsilon: 0.99
episode: 3/1000, score: 12, epsilon: 0.99
episode: 4/1000, score: 20, epsilon: 0.99


episode: 5/1000, score: 8, epsilon: 0.98
episode: 6/1000, score: 16, epsilon: 0.98
episode: 7/1000, score: 14, epsilon: 0.97
episode: 8/1000, score: 12, epsilon: 0.97
episode: 9/1000, score: 17, epsilon: 0.96


episode: 10/1000, score: 12, epsilon: 0.96
episode: 11/1000, score: 24, epsilon: 0.95
episode: 12/1000, score: 38, epsilon: 0.95
episode: 13/1000, score: 66, epsilon: 0.94
episode: 14/1000, score: 15, epsilon: 0.94


...

episode: 70/1000, score: 21, epsilon: 0.71
episode: 71/1000, score: 15, epsilon: 0.7
episode: 72/1000, score: 17, epsilon: 0.7
episode: 73/1000, score: 19, epsilon: 0.7


episode: 74/1000, score: 9, epsilon: 0.69
episode: 75/1000, score: 38, epsilon: 0.69
episode: 76/1000, score: 17, epsilon: 0.69
episode: 77/1000, score: 21, epsilon: 0.68


episode: 78/1000, score: 26, epsilon: 0.68
episode: 79/1000, score: 32, epsilon: 0.68
episode: 80/1000, score: 46, epsilon: 0.67
episode: 81/1000, score: 32, epsilon: 0.67


episode: 82/1000, score: 31, epsilon: 0.67


KeyboardInterrupt: 

In [ ]:
env.reset()
agent.load('best.h5')
for frame in range(TIME_MAX):
    env.render()
    act_values = agent.model.predict(state)
    action = np.argmax(act_values[0])            
    next_state, reward, done, _ = env.step(action)
    reward = reward if not done else -10
    next_state = np.reshape(next_state, [1, state_size])
    
    agent.remember(state, action, reward, next_state, done)
    
    state = next_state
    
    if done:
        print("score: {}" .format(frame))
        env.render(close=True)
        break

